In [124]:
import numpy as np
import xml.sax
import sys
import csv
import re
import string
import datetime
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk.stem.snowball import SnowballStemmer
from heapq import heapify, heappush, heappop
    

[nltk_data] Downloading package stopwords to /home/smriti/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [125]:
Titles= {}
STOPWORDS = set(stopwords.words('english')) 
URL_STOP_WORDS = set(["http", "https", "www", "ftp", "com", "net", "org", "archives", "pdf", "html", "png", "txt", "redirect"])
EXTENDED_PUNCTUATIONS = set(list(string.punctuation) + ['\n', '\t', ' '])
# INT_DIGITS = set(["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"])

inverted_FilePath= "./InvertedIndices/InvertedIndex_{}.txt"
invertedIndex_File= "InvertedIndex.txt"
maxDocs_indexed_atOnce= 80
indexed_untilNow =0
current_invertedFileIndex= 1
docId=1
Wiki_Dict= {}
Doc_Dict= {}

In [126]:
def kWayMerge():
    global current_invertedFileIndex
    numOfFiles= current_invertedFileIndex
    input_files= []
    
    for file_num in numOfFiles:
        input_files.append(open(inverted_FilePath.format(file_num), "r"))
        
    output_file= open(invertedIndex_File, "w")
    
        

In [127]:
def create_invertedIndex():
    global current_invertedFileIndex
    sorted(Wiki_Dict.items(), key=lambda t: t[0])
    f= open(invertedIndex_File, "w")
#     f = open(inverted_FilePath.format(current_invertedFileIndex), "w")
    for key, value in Wiki_Dict.items():
        line= key +':'+ value +"\n"
        f.write(line)
    f.close()
    current_invertedFileIndex +=1
    Wiki_Dict.clear()

In [128]:
# class Page:
def ProcessPage(pageID, content, index):
    for word in content:
        if word not in Doc_Dict:
            value= np.zeros(6, dtype= int)
            Doc_Dict[word]= value
            Doc_Dict[word][index] =1
        else:
            Doc_Dict[word][index] +=1
    
    if(index ==5):
#         print (Doc_Dict)
        for key, value in Doc_Dict.items():
            if key not in Wiki_Dict:                # pageID_totalcount:t1i2c3b4r5l6|pageID_totalcount:t1i2c3b4r5l6
                Wiki_Dict[key]= str(pageID)+ '-'+ 't'+ str(value[0])+ 'i'+ str(value[1])+ 'c'+ str(value[2])+ 'b'+ str(value[3])+ 'r'+ str(value[4])+ 'l'+ str(value[5])
            else:
                Wiki_Dict[key] += '|'+ str(pageID)+ '-'+ 't'+ str(value[0])+ 'i'+ str(value[1])+ 'c'+ str(value[2])+ 'b'+ str(value[3])+ 'r'+ str(value[4])+ 'l'+ str(value[5])
        
#         global maxDocs_indexed_atOnce, indexed_untilNow
    
#         if(maxDocs_indexed_atOnce == indexed_untilNow):
#             create_invertedIndex()
#             indexed_untilNow =0 
#         indexed_untilNow +=1
        Doc_Dict.clear()
        
def cleanText(text):
    stemmer= SnowballStemmer("english")
    text = re.sub(r'<(.*?)>','',text) #Remove tags if any
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text, flags=re.MULTILINE) #Remove Url
    text = re.sub(r'{\|(.*?)\|}', '', text, flags=re.MULTILINE) #Remove CSS
    text = re.sub(r'\[\[file:(.*?)\]\]', '', text, flags=re.MULTILINE) #Remove File
    text = re.sub(r'[.,;_()"/\'=]', ' ', text, flags=re.MULTILINE) #Remove Punctuaion
    text = re.sub(r'[~`!@#$%&-^*+{\[}\]()":\|\\<>/?]', ' ', text, flags=re.MULTILINE)
    text = text.strip().split()  
    text = [x for x in text if x not in STOPWORDS and x not in URL_STOP_WORDS]
    text = [stemmer.stem(word) for word in text]
    return text
    
def extract_pageDetails(text):
    lines= text.split('\n')
    line =0
    infoBox, category, body, references, links= [], [], [], [], []

    while line <len(lines):
        if "{{infobox" in lines[line]:
            openBrackets, closedBrackets= 0,0
            while line< len(lines):
                if "{{" in lines[line]:
                    openBrackets +=1
                if "}}" in lines[line]:
                    closedBrackets +=1

                infoBox.extend(cleanText(lines[line]))
                if(openBrackets == closedBrackets):
                    break
                line +=1
        elif "[[category:" in lines[line]:
            try:
                category.extend(cleanText(lines[line].split(':')[1].split(']]')[0]))
            except:
                category.extend(cleanText(lines[line].split(':')[1]))

        elif "==external links==" in lines[line] or "== external links ==" in lines[line]:
            line +=1
            while line< len(lines):
                if "*[" in lines[line] or "* [" in lines[line]:
                    link= ""
                    while line< len(lines) and "]" not in lines[line]:
                        link += lines[line]
                        line+=1
                    link += lines[line]
                    link = link.split('[')
                    if(len(link)> 1):
                        link= link[1].split(']')[0]
                        links.extend(cleanText(link))
                elif "[[category:" in lines[line]:
                    try:
                        category.extend(cleanText(lines[line].split(':')[1].split(']]')[0]))
                    except:
                        category.extend(cleanText(lines[line].split(':')[1]))
                else:
                    body.extend(cleanText(lines[line]))
                line+=1
                

        elif "==references==" in lines[line] or "== references ==" in lines[line]:
            line +=1
            while line< len(lines):
                if "==" in lines[line] or "[[category:" in lines[line]:
                    line -=1
                    break
                elif "{{cite" in lines[line] or "{{vcite" in lines[line]:
                    cite_title= lines[line].split("title=")
                    if(len(cite_title) >1):
                        cite_title= cite_title[1].split("|")[0]
                        references.extend(cleanText(cite_title))
                elif "{{" in lines[line] and "ref" not in lines[line]:
                    references.extend(cleanText(lines[line].split("{{")[1].split("}}")[0]))
                line+=1
        else:
            body.extend(cleanText(lines[line]))
        line +=1  
    
#     print (infoBox, category, body, references, links)
    return infoBox, category, body, references, links
#     return ' '.join(infoBox), ' '.join(category), ' '.join(body), ' '.join(references), ' '.join(links)

In [129]:
class Data:
    def __init__(self):
        self.docId = 0
        self.title=""
        self.text = ""
        self.infoBox= ""
        self.category= ""
        self.body= ""
        self.references= ""
        self.links= ""
        self.stemmer= SnowballStemmer("english")

    def set_data(self, docId, title, text):
        self.docId = docId
        self.title= title
        self.text = text

        self.infoBox, self.category, self.body, self.references, self.links = extract_pageDetails(self.text)
        
#         self.title= self.cleanText(self.title)
#         self.infoBox= self.cleanText(self.infoBox)
#         self.category= self.cleanText(self.category)
#         self.body= self.cleanText(self.body)
#         self.references= self.cleanText(self.references)
#         self.links= self.cleanText(self.links)
        
#         self.remove_numbers()
#         self.cleanText()
#         self.remove_punctuations()
#         self.strip_tokenize()
#         self.remove_stopWords()
#         self.stemming()
        
        ProcessPage(self.docId, self.title, 0)
        ProcessPage(self.docId, self.infoBox, 1)
        ProcessPage(self.docId, self.category, 2)
        ProcessPage(self.docId, self.body, 3)
        ProcessPage(self.docId, self.references, 4)
        ProcessPage(self.docId, self.links, 5)


    def remove_numbers(self):
        self.title = re.sub(r'\\d+', '', self.title)
        self.infoBox = re.sub(r'\\d+', '', self.infoBox)
        self.category = re.sub(r'\\d+', '', self.category)
        self.body = re.sub(r'\\d+', '', self.body)
        self.references = re.sub(r'\\d+', '', self.references)
        self.links = re.sub(r'\\d+', '', self.links)
    
    

    def remove_punctuations(self):
        self.title = self.title.translate(str.maketrans('', '', string.punctuation))
        self.infoBox = self.infoBox.translate(str.maketrans('', '', string.punctuation))
        self.category = self.category.translate(str.maketrans('', '', string.punctuation))
        self.body = self.body.translate(str.maketrans('', '', string.punctuation))
        self.references = self.references.translate(str.maketrans('', '', string.punctuation))
        self.links = self.links.translate(str.maketrans('', '', string.punctuation))

    def strip_tokenize(self):
        self.title= self.title.strip().split()
        self.infoBox= self.infoBox.strip().split()
        self.category= self.category.strip().split()
        self.body= self.body.strip().split()
        self.references= self.references.strip().split()
        self.links= self.links.strip().split()

    def remove_stopWords(self):
        self.title = [x for x in self.title if x not in STOPWORDS and x not in URL_STOP_WORDS]
        self.infoBox = [x for x in self.infoBox if x not in STOPWORDS and x not in URL_STOP_WORDS]
        self.category = [x for x in self.category if x not in STOPWORDS and x not in URL_STOP_WORDS]
        self.body = [x for x in self.body if x not in STOPWORDS and x not in URL_STOP_WORDS]
        self.references = [x for x in self.references if x not in STOPWORDS and x not in URL_STOP_WORDS]
        self.links = [x for x in self.links if x not in STOPWORDS and x not in URL_STOP_WORDS]


    def stemming(self):
        self.title = [self.stemmer.stem(line) for line in self.title]
        self.infoBox = [self.stemmer.stem(line) for line in self.infoBox]
        self.category = [self.stemmer.stem(line) for line in self.category]
        self.body = [self.stemmer.stem(line) for line in self.body]
        self.references = [self.stemmer.stem(line) for line in self.references]
        self.links = [self.stemmer.stem(line) for line in self.links]


In [130]:
class WikiHandler( xml.sax.ContentHandler ):
    def __init__(self):
        self.CurrentData = ""
        self.title = ""
        self.text = ""

    # Call when an element starts
    def startElement(self, tag, attributes):
        self.CurrentData = tag
        global docId
        if tag == "page":
            print (docId, end= " ")

    # Call when an elements ends
    def endElement(self, tag):
        global docId
        if self.CurrentData == "title":
            Titles[docId]= self.title
            print ("title: ", self.title)
        elif self.CurrentData == "text":
            page= Data()
            page.set_data(docId, self.title, self.text)
            docId +=1
        self.text= ""
        self.CurrentData = ""
#         if(docId ==50):
#             sorted(Wiki_Dict.items(), key=lambda t: t[0])
#             print (Wiki_Dict)
#             sys.exit()


    # Call when a character is read
    def characters(self, content):
        if self.CurrentData == "title":
            self.title = content.lower()
        elif self.CurrentData == "text":
            self.text += content.lower()


In [131]:
if ( __name__ == "__main__"):
    # create an XMLReader
    parser = xml.sax.make_parser()
    # turn off namepsaces
    parser.setFeature(xml.sax.handler.feature_namespaces, 0)

    # override the default ContextHandler
    Handler = WikiHandler()
    parser.setContentHandler(Handler)

    start = datetime.datetime.now()

    dump_data= "../Phase1/wikipedia.xml"
    parser.parse(dump_data)
    create_invertedIndex()

    end = datetime.datetime.now()
    secs  = (end-start).seconds
    hr = int(secs/(60*60))
    rm = int(secs%(60*60))
    mn = int(rm/60)
    rm=int(rm%60)
    secs = int(rm)
    print("Indexing Time : ",hr," hrs ",mn," mns",secs," secs")


1 title:  accessiblecomputing
2 title:  anarchism
3 title:  afghanistanhistory
4 title:  afghanistangeography
5 title:  afghanistanpeople
6 title:  afghanistancommunications
7 title:  afghanistantransportations
8 title:  afghanistanmilitary
9 title:  afghanistantransnationalissues
10 title:  assistivetechnology
11 title:  amoeboidtaxa
12 title:  autism
13 title:  albaniahistory
14 title:  albaniapeople
15 title:  aswemaythink
16 title:  albaniagovernment
17 title:  albaniaeconomy
18 title:  albedo
19 title:  afroasiaticlanguages
20 title:  artificallanguages
21 title:  abacus
22 title:  abalone
23 title:  abbadides
24 title:  abbess
25 title:  abbevillefrance
26 title:  abbey
27 title:  abbot
28 title:  abbreviations
29 title:  atlasshrugged
30 title:  artificiallanguages
31 title:  atlasshruggedcharacters
32 title:  atlasshruggedcompanies
33 title:  ayersmusicpublishingcompany
34 title:  africanamericanpeople
35 title:  adolfhitler
36 title:  abecedarians
37 title:  abel
38 title:  ab

309 title:  am
310 title:  automated alice/xii
311 title:  automated alice/xi
312 title:  automated alice/x
313 title:  automated alice/ix
314 title:  automated alice/viii
315 title:  automated alice/vi
316 title:  automated alice/vii
317 title:  automated alice/v
318 title:  automated alice/iv
319 title:  automated alice/ii
320 title:  automated alice/i
321 title:  automated alice/iii
322 title:  antigua and barbuda
323 title:  azincourt
324 title:  albert speer
325 title:  asteraceae
326 title:  apiaceae
327 title:  axon
328 title:  agma
329 title:  aramaic alphabet
330 title:  arguments for the existence of god
331 title:  american shot
332 title:  acute disseminated encephalomyelitis
333 title:  ataxia
334 title:  ambientcalculusonline
335 title:  abdul alhazred
336 title:  a priori and a posterior knowledge
337 title:  ada lovelace
338 title:  ambientcalculionline
339 title:  august derleth
340 title:  alps
341 title:  a priori and a posteriori knowledge
342 title:  albert camus
3

599 title:  aol
600 title:  adhd
601 title:  anno domini
602 title:  av
603 title:  amino group
604 title:  antony van leeuwenhook
605 title:  alcuin
606 title:  angilbert
607 title:  antony van leeuwenhoek
608 title:  amine
609 title:  adrian i
610 title:  april 29
611 title:  august 14
612 title:  absolute zero
613 title:  adiabatic process
614 title:  amide
615 title:  animism
616 title:  antonio vivaldi
617 title:  adrian ii
618 title:  adrian
619 title:  adrian iv
620 title:  adrian vi
621 title:  aare
622 title:  abgar
623 title:  abbotsford, scottish borders
624 title:  abraham
625 title:  abraxas
626 title:  absalom
627 title:  abydos
628 title:  abydos, egypt
629 title:  abydos (hellespont)
630 title:  august 15
631 title:  acacia sensu lato
632 title:  acapulco
633 title:  august 16
634 title:  alan kay
635 title:  apl (programming language)
636 title:  algol
637 title:  awk
638 title:  alzheimers disease
639 title:  ascorbic acid
640 title:  asgard
641 title:  apollo program

903 title:  asa
904 title:  aquinas
905 title:  actium
906 title:  amide hydrolysis
907 title:  amway
908 title:  adam smith
909 title:  antoine laurent lavoisier
910 title:  antoine lavoisier
911 title:  a roll
912 title:  hermann kolbe
913 title:  april 18
914 title:  april 23
915 title:  amitabh bachchan
916 title:  air pollution
917 title:  antarctic-environmental protocol
918 title:  allomorph
919 title:  american bias
920 title:  allophone
921 title:  affix
922 title:  allegory
923 title:  amazon river
924 title:  allotropy
925 title:  agathocles of syracuse
926 title:  economy of alberta
927 title:  augustin-louis cauchy
928 title:  archimedes
929 title:  alternative medicine
930 title:  archimedean solid
931 title:  antiprism
932 title:  ancient greeks
933 title:  natural history of africa
934 title:  geography of africa
935 title:  africa/history
936 title:  approval voting
937 title:  aromatic compound
938 title:  arizona state university
939 title:  april 14
940 title:  asto

1192 title:  acts
1193 title:  apicomplexa
1194 title:  argentine cuisine
1195 title:  abattoir
1196 title:  april 8
1197 title:  ad hominem
1198 title:  a fortiori
1199 title:  argumentum ad hominum
1200 title:  analysis of algorithms
1201 title:  ælle of sussex
1202 title:  atari
1203 title:  afghan
1204 title:  acadia university
1205 title:  steel-string acoustic guitar
1206 title:  antipope john xxiii
1207 title:  antonio salieri
1208 title:  cobble hill tunnel
1209 title:  annapolis valley
1210 title:  analgesic
1211 title:  amplify
1212 title:  abiotic stress
1213 title:  accusative case
1214 title:  apostolic succession
1215 title:  anglican
1216 title:  astounding magazine
1217 title:  analog magazine
1218 title:  analog magazine
1219 title:  list of anglo-saxon monarchs and kingdoms
1220 title:  chemistry of ascorbic acid
1221 title:  asyncronous transfer mode
1222 title:  asynchronous digital subscriber line
1223 title:  afc ajax
1224 title:  arthur eddington
1225 title:  app

...

18736 title:  stoichiometry
18737 title:  outline of statistics
18738 title:  outline of sociology
18739 title:  outline of sculpture
18740 title:  second law
18741 title:  sickle cell anaemia
18742 title:  singularity
18743 title:  defamation
18744 title:  sweetbread
18745 title:  principality of sealand
18746 title:  sweetener (disambiguation)
18747 title:  susan faludi
18748 title:  oswald of northumbria
18749 title:  steve lacy (saxophonist)
18750 title:  sabah
18751 title:  sumatra
18752 title:  sarkel
18753 title:  streaming media
18754 title:  session initiation protocol
18755 title:  squatting
18756 title:  south holland
18757 title:  subordinate
18758 title:  screaming lord sutch
18759 title:  protein secondary structure
18760 title:  sabermetrics
18761 title:  smooth er
18762 title:  space quest
18763 title:  mercy rule
18764 title:  sumbawa
18765 title:  spike milligan
18766 title:  sylvia plath
18767 title:  shania twain
18768 title:  session description protocol
18769 titl

19019 title:  submarine recycling
19020 title:  sansad
19021 title:  security through obscurity
19022 title:  snuff film
19023 title:  software testing
19024 title:  ship-submarine recycling program
19025 title:  states of matter
19026 title:  situational dynamics
19027 title:  social psychology
19028 title:  shaolin monastery
19029 title:  sycorax (disambiguation)
19030 title:  seymour cray
19031 title:  signature block
19032 title:  sig block
19033 title:  semantics
19034 title:  semantic network
19035 title:  social dynamics
19036 title:  stockholm bloodbath
19037 title:  schrodingers equation
19038 title:  sexual practices
19039 title:  ssl
19040 title:  signals intelligence
19041 title:  semantic web
19042 title:  soviet submarine k-219
19043 title:  soviet submarine k-8
19044 title:  soviet submarine k-19
19045 title:  list of seleucid rulers
19046 title:  super bowl i
19047 title:  super bowl ii
19048 title:  super bowl iv
19049 title:  super bowl v
19050 title:  super bowl vi
1

19293 title:  seabee
19294 title:  skyscraper
19295 title:  sagas of icelanders
19296 title:  september 11 terrorist attack
19297 title:  staind
19298 title:  saddam hussein
19299 title:  sonja henie
19300 title:   environmental policy project
19301 title:  abbey of saint gall
19302 title:  secondary education
19303 title:  serotonin syndrome
19304 title:  sustainable development
19305 title:  six day war
19306 title:  scientific american
19307 title:  serotonin specific reuptake inhibitors
19308 title:  serotonin-specific reuptake inhibitor
19309 title:  siouxsie and the banshees
19310 title:  simula
19311 title:  snobol
19312 title:  statistical physics
19313 title:  side effect (computer science)
19314 title:  superheterodyne
19315 title:  science fiction writers
19316 title:  list of science fiction editors
19317 title:  square-free integer
19318 title:  special function
19319 title:  sentinel (comics)
19320 title:  sebastian shaw
19321 title:  savage land
19322 title:  stephen sch

19555 title:  triassic
19556 title:  titanic thompson
19557 title:  the shockwave rider
19558 title:  turkish language
19559 title:  the thin blue line (disambiguation)
19560 title:  the americans with disabilites act of 1990
19561 title:  the americans with disabilites act of 1990/findings and purposes
19562 title:  the americans with disabilites act of 1990/title iii
19563 title:  the americans with disabilites act of 1990/definitions
19564 title:  the short story
19565 title:  the shining (novel)
19566 title:  taxi driver
19567 title:  theory of relativity
19568 title:  telephone
19569 title:  telia company
19570 title:  telefónica
19571 title:  the silence of the lambs (film)
19572 title:  the matrix
19573 title:  telegraph
19574 title:  telegraphy
19575 title:  transistor
19576 title:  time
19577 title:  tone
19578 title:  tifinagh
19579 title:  turkic languages
19580 title:  the sound of music
19581 title:  the venerable bede
19582 title:  trip hop
19583 title:  traitor
19584 tit

19791 title:  tensor/alternate
19792 title:  the hobbit
19793 title:  the lord of the rings/one ring
19794 title:  tax freedom day
19795 title:  tax
19796 title:  transhumanism
19797 title:  tardis
Indexing Time :  0  hrs  23  mns 21  secs
